# グラフ探索アルゴリズムI 第2回

前回は重複検知を扱わない単純なアルゴリズムを紹介した。
また、重複検知を行わないことで探索の効率が悪くなる場合があることを示した。
今回はまず、重複検知を含む、最良優先探索の基本形を紹介する。

## 前回までの補足

用語
- 次数 (degree): 一つの節点から出ている枝の本数。あまり講義中で使っていないが…

デバッグ方法の参考
- Jupyter Notebook 上の python のデバッグについて  
https://colab.research.google.com/drive/1di0EBQ5FaGME82JOLDP0kjaerk1fqakk?usp=sharing#scrollTo=0d6492d0
- 可能なら VScode などを用いることも推奨する。
  Jupyter Notebook を python 形式でダウンロードして実行可能（なように書いたつもりですが、誤りがあったらご連絡ください）。

コードの例示についてフィードバックをください。例えば
1. classなどを使わないpythonコードがあれば疑似コード無しでよい
1. 疑似コード + class を使った python が良い
1. 疑似コード + classなどを使わないpythonコード

その他何でもご意見お待ちしています。
合理的な努力で可能な範囲で対応します。


## 最良優先探索 (best-first search)

何らかのスコア関数 $f$ が最小の節点から順番に探索するアルゴリズムを最良優先探索と呼ぶ。

重要なデータ構造
- **open list**: 探索中の節点 (Node) のリスト。
  リストという名称が付いているが、実際にはスコアが最小の節点を返す機能を持つ Priority Queue を使うべきである。
  frontier と呼ばれることもある。
- **closed list**: 探索が終了した節点の情報を保持する。
  節点のstateをkeyとして値 (この場合は Node) を参照することができる。
  これもリストという名称が付いているが、通常は辞書 (ハッシュ表, hash table) を用いる。

これらを用いた最良優先探索の疑似コードと Python 風擬似コードを以下に示す。
節点のデータ構造 Node は状態 `state` (この場合は節点の番号) と `cost` (前回 dist と呼んだがそれの一般化) を含むとする。
closed_list は `state` を key とするハッシュ表で、 key に対応するデータが記録されているかどうか調べることができる。
もし `state` に対応する節点が含まれていれば `closed_table[state]` は該当の Node を返す。
そうでない場合には、該当の Node が含まれていないことを示す値を返す (`failure` または `None`)。
発見できた場合には節点を返し、そうでない場合には失敗を示す値を返す (`failure` または `None`)。
（もし経路が存在するなら発見できるはず。)

### Best-First Search 疑似コード

```
引数
prob: graph, start, goal
f: score function
返値
発見された節点又は failure

fun BestFirstSearch (prob, f) {
  open_list <- start 節点のみを含む priority queue, 関数fでソートされている
  closed_list <- start 節点のみを含む hash table
  while (open_list is not empty) {
    node <- Pop(open_list)
    if (prob.goal == node) return node
    for each child in Expand(prob, node) {
      st <- child.state
      if (st is not in closed_list or child.dist < closed_list[st].dist) {
        closed_list[s] <- child
        open_list.add(child)
      }
  return failure
```

### Best-First Search Python 風擬似コード (fが未定義なので実行不可能)

```Python
def BestFirstSearch (prob, f):
    start_node = Node(state=prob.start)
    open_queue = [start_node]
    closed_table = [ start_node ]
    while (open_list is not empty):
        node = pop(open_list)
        if (prob.goal == node):
            return node
        children = Expand(prob, node)
        for child in children:
            st = child.state
            if (s is not in closed_list or child.path_cost < closed[st]):
                open_queue.add(child)
    return None
```

### 優先度付きキュー priority queue

最小 (最大) の要素が常に先頭にあるキューのことを言う。
実現するアルゴリズムは多数あるが、通常は以下の操作が小さい計算量でできることが期待される。
- 先頭の要素の参照: $O(1)$
- 先頭の要素の削除: $O(log n)$ 以下
- 新たな要素の挿入: $O(log n)$ 以下

binary heap は priority queue を実現する方法の一つである。
以下のPython実装では Python ライブラリの heapq を用いる。

## ダイクストラ法 Dijkstra's Algorithm

上記の例では関数fが明示されていないので当然このままでは実行不可能である。
単純なアイデアとして、現在判明しているstart節点からの距離をそのまま$f$とすることが考えられる。
この関数を慣例的に $g$と呼ぶ。ある節点 $n$ に対して $g(n) = node.dist$ と定義して
上記コードで $f=g$ とした物がダイクストラ法である。
ダイクストラ法は経路の重みが全て非負である場合に、開始節点から
ゴール節点までの最短経路を発見するアルゴリズムである。
また、終了条件を変更することで開始節点から全ての節点への最短経路を求めることもできる。
この問題を正確には単一始点最短経路 (Single Source Shortest Path, SSSP) と呼ぶ。

以下に Python による実装例を示す。

<img src="fig1-1.png" width=200>  
グラフ1

In [9]:
import numpy as np
import heapq

graph_array = np.array(
#     [S, a, b, c, d, G]
    [[ 0, 2, 3, 0, 5, 0], # S
     [ 0, 0, 0, 4, 0, 0], # a
     [ 0, 0, 0, 0, 4, 0], # b
     [ 0, 0, 0, 1, 0, 2], # c
     [ 0, 0, 0, 0, 0, 5], # d
     [ 0, 0, 0, 0, 0, 0]] # G
)


class SSSP_simple:
    def __init__(self, s, g, graph_array):
        self.start = s # node S
        self.goal = g # node G
        self.graph = graph_array

        
class Node:
    def __init__(self, state, cost, parent_state):
        self.st = state
        self.cost = cost
        self.par = parent_state

    def __lt__(self, other):
        '''
        Node 同士の大小比較を定義します (python 3)
        priority queue で自動的に使われます
        これが関数 f の定義となります
        この例では距離をそのまま使っています
        '''
        return self.cost < other.cost

    # 以下は、printでNodeを表示するための関数です
    def __str__(self):
        return "id:" + str(self.st) + " cost=" + str(self.cost) + " par=" + str(self.par)

    def __repr__(self):
        return f'Node({self.st}, {self.cost}, {self.par})'


class Dijkstra:
    def __init__(self, problem):
        self.prob = problem
        self.node_num = self.prob.graph.shape[0]
  
    def expand(self, node):
        # return list of child nodes
        child_nodes = []
        for c in range(0, self.node_num):
            d = self.prob.graph[node.st][c]
            if d != 0:
                child_nodes.append( Node(c, node.cost + d, node.st) )
        return child_nodes
    
    def search(self):
        # this is the start node
        # state = 0 (Start node id)
        # cost = 0 (0 distance from the start node)
        # parent state = -1 (means there is no parent for this node)
        start_node = Node(self.prob.start, 0, -1)
        
        if (start_node.st == self.prob.goal): # checking exceptional case, start == goal
            return start_node

        # open list 定義
        # Python の heap はデフォルトでは tuple の最初の要素から比較するので距離を先頭にしている
        open_queue = [ start_node ]
        heapq.heapify(open_queue)
        
        # closed list 定義
        closed_list = {}

        print("open list: " + str(open_queue) + "\nclosed list: " + str(closed_list))
        
        while len(open_queue) != 0:
            n = heapq.heappop(open_queue)
            #print(n.__repr__,'----------------------------------------')
            if n.st == self.prob.goal:
                return n
            child_nodes = self.expand(n)
            for c in child_nodes:
                if c.st not in closed_list or c.cost < closed_list[c.st].cost:
                    closed_list[c.st] = c
                    heapq.heappush(open_queue, c)
            print("open list: " + str(open_queue) + "\nclosed list: " + str(closed_list))
            # print('=====================================================')
        return None
    
prob1 = SSSP_simple(0, 5, graph_array)
solver = Dijkstra(prob1)
goal = solver.search()

print(goal)

open list: [Node(0, 0, -1)]
closed list: {}
open list: [Node(1, 2, 0), Node(2, 3, 0), Node(4, 5, 0)]
closed list: {1: Node(1, 2, 0), 2: Node(2, 3, 0), 4: Node(4, 5, 0)}
open list: [Node(2, 3, 0), Node(4, 5, 0), Node(3, 6, 1)]
closed list: {1: Node(1, 2, 0), 2: Node(2, 3, 0), 4: Node(4, 5, 0), 3: Node(3, 6, 1)}
open list: [Node(4, 5, 0), Node(3, 6, 1)]
closed list: {1: Node(1, 2, 0), 2: Node(2, 3, 0), 4: Node(4, 5, 0), 3: Node(3, 6, 1)}
open list: [Node(3, 6, 1), Node(5, 10, 4)]
closed list: {1: Node(1, 2, 0), 2: Node(2, 3, 0), 4: Node(4, 5, 0), 3: Node(3, 6, 1), 5: Node(5, 10, 4)}
open list: [Node(5, 8, 3), Node(5, 10, 4)]
closed list: {1: Node(1, 2, 0), 2: Node(2, 3, 0), 4: Node(4, 5, 0), 3: Node(3, 6, 1), 5: Node(5, 8, 3)}
id:5 cost=8 par=3


### ダイクストラ法の動作と正しさ

最初に述べたとおり、同じ節点の再展開 (re-expand) は無駄なので可能な限り避けたい。
ダイクストラ法は再展開しない、つまり同じ節点は最大でも1回しか展開されないが、なぜこれで正しいのか念のため簡単に確認しよう。

open_list から pop された節点 $n$ に注目しよう。
節点 $n$ についてのスタート節点からの距離 $n.cost$ を下回る長さの経路は
その時点で全て探索済みのはずである。
つまり open_list の先頭で pop された時点で、判明している距離が真の最短距離である。
よって節点 $n$ の探索は終了してよい。

#### 理解度チェック

1. 上記のコードのごく一部を変更すると、スタート節点から全ての節点への最短経路を求めることができる。
   どこを変更すれば良いか？
1. 上記の正しさが成り立たなくなるケースはどのようなものか。

### ダイクストラ法の計算量

節点数を N, 枝数を E として考えよう。
最初の疑似コードの中でまず
```
  while (open_list is not empty)
```
の部分は全節点について繰り返される。よってここは $O(V)$。
また open_list から先頭を取り出す以下の部分はどうだろうか。
```
    node <- Pop(open_list)
```
ここは、もし通常の priority queue が使われていれば $O(log V)$ で実行可能である。
以下の open_list への追加も同様である。
```
        open_list.add(child)
```
最後に
```
    for each child in Expand(prob, node) {
```
の部分はどうだろうか。これは最終的には各節点の枝を1回たどるので、全て合計すると $O(E)$である。
以上を合計して $ O(E + V log V)$ がダイクストラ法の計算量となる。

```
fun BestFirstSearch (prob, f) {
  open_list <- start 節点のみを含む priority queue, 関数fでソートされている
  closed_list <- start 節点のみを含む hash table
  while (open_list is not empty) {
    node <- Pop(open_list)
    if (prob.goal == node) return node
    for each child in Expand(prob, node) {
      st <- child.state
      if (st is not in closed_list or child.dist < closed_list[st].dist) {
        closed_list[s] <- child
        open_list.add(child)
      }
  return failure
```

なお細かい話になるが、教科書によっては priority queue に Fibonacci heap を使うと書かれていることがある。
しかし Fibonacchi heap は計算量のオーダーは良いが実際にはほとんどの場合にその他の heap より遅い。
通常は二分ヒープで十分である。ある条件を満たせば基数ヒープ (radix heap) も良い。

### ハッシュ表の必要性

今回はハッシュ表を用いた実装を示した。
しかしハッシュ表は常に必要なわけではない。
上で例に示した小さなグラフであれば、節点数と同じサイズの配列に記録しても問題ない。
（以下の課題を参照。）

### 重複検出 (duplicate detection) の細かい話

先ほどのダイクストラ法の出力を見てみよう。
closed list には重複はない。しかし open list には重複がある（なので上記の計算量の解析も少し不正確である。）
これは、上記のコード（疑似コードも）重複検出を行うタイミングを節点が展開されたタイミングとしているからである。
open list に追加するタイミングで重複検出を行うことはもちろん可能で、
そうすれば計算量の解析も上述の通りになる。
open list に追加するタイミングで重複検出を行う方法を**即時重複検出 (immediate duplicate detection)** と
呼ぶことがある。
対して上記のコードのように遅れたタイミングで行う方法を**遅延重複検出 (delayed duplicate detection)** と呼ぶ。
はっきりした優劣はないが、ノードの重複が少ない（合流の頻度が低い）問題や重複検出に時間がかかる場合に
遅延重複検出が使われる。


## A*探索 (A* search) の紹介

ダイクストラ法は手がかりがない場合には実際に使用されることもあり
最短経路探索の代表的な手法であるが、
たとえば2Dグリッドのような問題の場合にはもっと頭の良い方法はないだろうか。

人間ならば単純な直感に基づいてまずゴールのある方向へ向かうはずだろう。
このような直感をアルゴリズムを活用したいと考えるのが自然である。
問題の特徴をもとにノードの有望さを推定する**ヒューリスティック関数 （heuristic function)** を使えば
うまくいけば性能を向上させることができる。
またこれを用いた探索を**ヒューリスティック探索 (heuristic search)** と言う。

ダイクストラ法の説明で、節点 $n$ のスコアをスタート節点から $n$ までの距離 $g(n)$と定義した。
これにヒューリスティック関数 $h$ を定義し、節点 $N$ のスコアを $ f(n) = g(n) + h(n) $ と
定義することが考えられる。
ダイクストラ法はスタート節点から同じ程度の距離の節点を open list に入れて探索する性質があるが、
$ h(n) $ を適切に定義すればゴール節点に近い節点を優先して探索させることができる。

では、ヒューリスティック関数はどのように作れば良いだろうか？
基本となる考え方は、$h(n)$ は節点$n$からゴール節点までの最短距離の推定だということだ。
節点$n$からゴールまでの正しい最短距離を $h^*(n)$ とすると、$h(n) = h^*(n)$ なら
**完璧なヒューリスティック (perfect heuristic)** ということになる。
実際にはよほど簡単な問題でない限り完璧なヒューリスティックは得られないが、
近いほど探索の効率が良いことが多い。
探索の効率が良いというのは、節点を展開する回数が少ないということである。

詳しいことは次回説明するが、$h(n)$が$h^*(n)$を超えない場合、
$h(n)$を使ったヒューリスティック探索によって、常に最適解を得ることができる。
このようなヒューリスティック関数を**許容的なヒューリスティック (admissible heuristic)**
と言い、またその探索アルゴリズムを**A*探索 (A* search, A-star search)** と言う。

例えば2Dグリッド上の最短経路探索に対して、ゴールまでのマンハッタン距離は良い
ヒューリスティック関数になる。
（もし全く障害物がなければ完璧なヒューリスティックである。）
実際にダイクストラと比べて圧倒的に効率が良くなる。


## 第2回 課題:

(発展課題) は単位取得に必須ではない。できたところまでで良いので提出すること。
締切は次々回の講義の前まで。

1. open list にただの配列を使うこともできる。その場合のダイクストラ法の計算量はどうなるか。上記と同様に解析せよ。
1. closed list にハッシュ表ではなくただの配列（場合によっては多次元配列）を使うとどのような問題があるか。例えば前回説明した 8-puzzle やハノイの塔の場合はどうか。15-puzzleやハノイの塔で円盤が増えたらどうだろうか。逆に配列を使って良いのはどのような場合か。
1. coding: 以下のコードの空白部分を実装し、2Dグリッド上のダイクストラ法を実装せよ。
    - (発展課題) 最終的に発見された経路を表示せよ (テキストで良い)。
    - (発展課題) この場合はハッシュ表を使わずに2次元配列を使っても良い。そのように実装を変更せよ。
1. (発展課題) coding: 以下のコードに追加して、マンハッタン距離をヒューリスティック関数として使う2Dグリッド上のA*探索を実装せよ。以下のヒントを手がかりとして良い。
    - コードの大部分はそのまま使えるが MazeNode と MazeDijkstra の定義は共に変更するのが自然である。
      class MazeNode2 や class MazeAStar を定義しても良い。
    - MazeNode の $cost$ を $g$ と $h$ の二つに変更し、両方を記録せよ。
    - また、open list は $f+g$ でソートされるように `__lt__` を変更せよ。

### 参考
- (外部リンク) 
    AizuOnlineJudge の以下の問題を解いてみる。  
    https://ja.wikipedia.org/wiki/AizuOnlineJudge
    https://judge.u-aizu.ac.jp/onlinejudge/description.jsp?id=ALDS1_12_B&lang=ja
- Python の priority queue:  
  上記のコードの例の様に heapq を使ってリストを priority queue として扱うことができる。比較方法については、class なら __lt__ 関数を定義して置けば動作する。(less than のこと, python 3 の機能)。あるいは、class ではなく tuple を使った場合、先頭の要素から比較して行く。
- C++ での priority queue:  
  ライブラリの std::prioritye_queue を使う方法が簡単だが、class と template の知識が必要になる。今回の課題なら配列を使っても良い。
- Python/C++によるアルゴリズムの実装例はweb上に大量に存在する。参考にしても良いが、何を参考にしたのか明記すること。

------------------------------------------------------------
ここに課題1, 2 の回答を記入せよ。課題3以降については、動作が検証できるソースコードを何らかの方法で提出すれば良い。ただしコードのコメントは歓迎する。

------------------------------------------------------------

### 課題1:
open list にただの配列を使うこともできる。その場合のダイクストラ法の計算量はどうなるか。上記と同様に解析せよ。


#### answer:
while (open_list is not empty)
ここは  𝑂(𝑉)

node <- Pop(open_list)
ここはopen_list にただの配列が使われていれば  𝑂(𝑉)

for each child in Expand(prob, node) {
ここは 𝑂(𝐸) 

以上を合計して  𝑂(𝐸+𝑉^2)  

-----------------------------

### 課題2:
closed list にハッシュ表ではなくただの配列（場合によっては多次元配列）を使うとどのような問題があるか。
例えば前回説明した 8-puzzle やハノイの塔の場合はどうか。15-puzzleやハノイの塔で円盤が増えたらどうだろうか。逆に配列を使って良いのはどのような場合か。

#### answer:
For each modification of the closed list, we need to find the target node for comparison, 
then delete the original node from the closed list, and then add a new node.
Otherwise, there will be multiple paths and costs for the same node.

For 8-pluzzle and Tower of Hanoi, the same number or disc will appear multiple times.
    
The advantage of using list: All path information of each node can be recorded in closed list.

------------------------------------------------------------------------------------

(以下のコードの解説をします。また質問を受け付けます。)

### 課題3：

In [12]:
import heapq

class Maze2D:
    def __init__(self):
        self.x_size = 20
        self.y_size = 20
        self.maze = """********************
*                  *
*                  *
*     S            *
*                  *
*                  *
*            *     *
*            *     *
*            *     *
*            *     *
*            *     *
*  ***********     *
*                  *
*           G      *
*                  *
*                  *
*                  *
*                  *
*                  *
********************
"""

    def get(self, x, y):
        if x < 0 or self.x_size <= x or y < 0 or self.y_size <= y:
            raise ValueError
        return self.maze[x + (self.x_size + 1) * y]
    

    def set(self,x,y,a):
        if x < 0 or self.x_size <= x or y < 0 or self.y_size <= y:
            raise ValueError
        self.maze=self.maze[0:(x + (self.x_size + 1) * y)]+a+self.maze[(x + (self.x_size + 1) * y)+1:]
    
    
    def check_bounds(self, x, y):
        if x < 0 or self.x_size <= x or y < 0 or self.y_size <= y:
            return False
        else:
            return True
        
    def is_blank(self, x, y):
        if x < 0 or self.x_size <= x or y < 0 or self.y_size <= y:
            raise ValueError
        if self.maze[x + (self.x_size + 1) * y] == ' ':
            return True
        else:
            return False
    
    def is_wall(self, x, y):
        if x < 0 or self.x_size <= x or y < 0 or self.y_size <= y:
            raise ValueError
        if self.maze[x + (self.x_size + 1) * y] == '*':
            return True
        else:
            return False

    def is_start(self, x, y):
        if x < 0 or self.x_size <= x or y < 0 or self.y_size <= y:
            raise ValueError
        if self.maze[x + (self.x_size + 1) * y] == 'S':
            return True
        else:
            return False
        
    def is_goal(self, x, y):
        if x < 0 or self.x_size <= x or y < 0 or self.y_size <= y:
            raise ValueError
        if self.maze[x + (self.x_size + 1) * y] == 'G':
            return True
        else:
            return False
        
maze = Maze2D()
print(maze.is_start(6, 3))
print(maze.is_goal(12, 13))

class SimpleMaze:
    def __init__(self, s, g, maze):
        self.start = s
        self.goal = g
        self.maze = maze

prob2 = SimpleMaze((6, 3), (12, 13), maze)
        
class MazeNode:
    def __init__(self, state, cost, parent_state):
        self.st = state # Hint: naturally, this will be a tuple of (x, y)
        self.cost = cost
        self.par = parent_state

    def __lt__(self, other):
        '''
        Node 同士の大小比較を定義します (python 3)
        priority queue で自動的に使われます
        これが関数 f の定義となります
        この例では距離をそのまま使っています
        '''
        return self.cost < other.cost

    # 以下は、printでNodeを表示するための関数です
    def __str__(self):
        return "id:" + str(self.st) + " cost=" + str(self.cost) + " par=" + str(self.par)

    def __repr__(self):
        return f'Node({self.st}, {self.cost}, {self.par})'


class MazeDijkstra:
    def __init__(self, problem):
        self.prob = problem
  
    def expand(self, node):
        # return list of child nodes
        child_nodes = []
        '''
        ここを実装してください
        上下左右の4方向に移動できるかどうかチェックして、壁がなければ子節点を追加しましょう
        '''
        x,y=node.st
        if self.prob.maze.is_blank(x+1,y) or  self.prob.maze.is_goal(x+1,y):
            child_nodes.append(MazeNode((x+1,y),node.cost+1,node))
        if self.prob.maze.is_blank(x-1,y) or self.prob.maze.is_goal(x-1,y):
            child_nodes.append(MazeNode((x-1,y),node.cost+1,node))
        if self.prob.maze.is_blank(x,y+1) or self.prob.maze.is_goal(x,y+1):
            child_nodes.append(MazeNode((x,y+1),node.cost+1,node))
        if self.prob.maze.is_blank(x,y-1) or self.prob.maze.is_goal(x,y-1):
            child_nodes.append(MazeNode((x,y-1),node.cost+1,node))
        return child_nodes
    
    def search(self):
        # this is the start node
        # state = (6, 3) # 状態が x, y の2次元なことに注意
        # cost = 0 (0 distance from the start node)
        # parent state = -1 (means there is no parent for this node)
        start_node = MazeNode(self.prob.start, 0, (-1, -1))
        
        if (start_node.st == self.prob.goal): # checking exceptional case, start == goal
            return start_node

        # open list 定義
        # Python の heap はデフォルトでは tuple の最初の要素から比較するので距離を先頭にしている
        open_queue = [ start_node ]
        heapq.heapify(open_queue)
        
        # closed list 定義
        closed_list = {}

        # print("open list: " + str(open_queue) + "\nclosed list: " + str(closed_list))

        '''
        余裕があれば最後に経路を表示してください
        '''
        while len(open_queue) != 0:
            n = heapq.heappop(open_queue)
            if n.st == self.prob.goal:
                return n
            child_nodes = self.expand(n)
            #print(child_nodes)
            for c in child_nodes:
                if c.st not in closed_list or c.cost < closed_list[c.st].cost:
                    closed_list[c.st] = c
                    heapq.heappush(open_queue, c)
                    # self.prob.maze.set(c.st[0],c.st[1],'0')
            # print(self.prob.maze.maze)
            #print("open list: " + str(open_queue) + "\nclosed list: " + str(closed_list))
            #print('-------------------')
        return closed_list[self.prob.goal]
    
    def search_by_list(self):
        # ハッシュ表を使わずに2次元配列を使って
        start_node = MazeNode(self.prob.start, 0, (-1, -1))
        
        if (start_node.st == self.prob.goal): # checking exceptional case, start == goal
            return start_node

        # open list 定義
        # Python の heap はデフォルトでは tuple の最初の要素から比較するので距離を先頭にしている
        open_queue = [ start_node ]
        heapq.heapify(open_queue)
        
        # closed list 定義
        closed_list = [[MazeNode((i,j),99999,(-1,-1)) for i in range(self.prob.maze.y_size)] for j in range(self.prob.maze.x_size)]
        # print(closed_list)
        closed_list[self.prob.start[0]][self.prob.start[1]]=start_node
        print(closed_list[self.prob.start[0]][self.prob.start[1]])

        # print("open list: " + str(open_queue) + "\nclosed list: " + str(closed_list))

        '''
        余裕があれば最後に経路を表示してください
        '''
        while len(open_queue) != 0:
            n = heapq.heappop(open_queue)
            if n.st == self.prob.goal:
                return n
            child_nodes = self.expand(n)
            #print(child_nodes)
            for c in child_nodes:
                if c.cost < closed_list[c.st[0]][c.st[1]].cost:
                    closed_list[c.st[0]][c.st[1]] = c
                    heapq.heappush(open_queue, c)
        return closed_list[self.prob.goal]
        

solver1 = MazeDijkstra(prob2)

# #use hash table: search()
# goal = solver1.search()

# #use 2D-list
goal= solver1.search_by_list()

# print(goal)
# print('-------------')




s=str(goal)
result_list,result=[],[prob2.goal]
for i in s.split('id:'):
    result_list.append(i.split('cost')[0][1:-2])
for i in result_list[2:-1]:
    x=tuple(map(int,i.split(',')))
    result.append(x)
    prob2.maze.set(x[0],x[1],'o')
    #print(prob2.maze.maze)
result.append(prob2.start)
print(result)
print("cost=",goal.cost)
print(prob2.maze.maze)



# with open('result.txt','w') as f:
#     f.write(str(result))
#     f.write("\ncost=")
#     f.write(str(goal.cost))
#     f.write("\n")
#     f.write(prob2.maze.maze)

True
True
id:(6, 3) cost=0 par=(-1, -1)
[(12, 13), (12, 12), (13, 12), (14, 12), (14, 11), (14, 10), (14, 9), (14, 8), (14, 7), (14, 6), (14, 5), (14, 4), (14, 3), (13, 3), (12, 3), (11, 3), (10, 3), (9, 3), (8, 3), (7, 3), (6, 3)]
cost= 20
********************
*                  *
*                  *
*     Soooooooo    *
*             o    *
*             o    *
*            *o    *
*            *o    *
*            *o    *
*            *o    *
*            *o    *
*  ***********o    *
*           ooo    *
*           G      *
*                  *
*                  *
*                  *
*                  *
*                  *
********************



--------------------------------------------------------

### 課題4:
(発展課題) coding: 以下のコードに追加して、マンハッタン距離をヒューリスティック関数として使う2Dグリッド上のA*探索を実装せよ。以下のヒントを手がかりとして良い。  
    - コードの大部分はそのまま使えるが MazeNode と MazeDijkstra の定義は共に変更するのが自然である。
      class MazeNode2 や class MazeAStar を定義しても良い。  
    - MazeNode の $cost$ を $g$ と $h$ の二つに変更し、両方を記録せよ。  
    - また、open list は $f+g$ でソートされるように `__lt__` を変更せよ。

In [19]:
def manhattan(node1,node2):
    return abs(node1[0]-node2[0])+abs(node1[1]-node2[1])

class MazeNode2():
    def __init__(self, state, cost, parent_state):
        self.st = state # Hint: naturally, this will be a tuple of (x, y)
        self.cost = cost
        self.manh = 0
        self.par = parent_state

    def __lt__(self, other):
        return (self.cost + self.manh) < (other.cost + other.manh)

    # 以下は、printでNodeを表示するための関数です
    def __str__(self):
        return "id:" + str(self.st) + " cost=" + str(self.cost) + " manhattan="+ str(self.manh) + " par=" + str(self.par)

    def __repr__(self):
        return f'Node({self.st}, {self.cost},{self.manh},{self.par})'
    
    

class Mazestar():
    def __init__(self, problem):
        self.prob = problem
  
    def expand(self, node):
        # return list of child nodes
        child_nodes = []
        '''
        ここを実装してください
        上下左右の4方向に移動できるかどうかチェックして、壁がなければ子節点を追加しましょう
        '''
        x,y=node.st
        if self.prob.maze.is_blank(x+1,y) or  self.prob.maze.is_goal(x+1,y):
            child_nodes.append(MazeNode2((x+1,y),node.cost+1,node))
        if self.prob.maze.is_blank(x-1,y) or self.prob.maze.is_goal(x-1,y):
            child_nodes.append(MazeNode2((x-1,y),node.cost+1,node))
        if self.prob.maze.is_blank(x,y+1) or self.prob.maze.is_goal(x,y+1):
            child_nodes.append(MazeNode2((x,y+1),node.cost+1,node))
        if self.prob.maze.is_blank(x,y-1) or self.prob.maze.is_goal(x,y-1):
            child_nodes.append(MazeNode2((x,y-1),node.cost+1,node))
        return child_nodes
    
    def search(self):
        start_node = MazeNode2(self.prob.start, 0, (-1, -1))
        
        if (start_node.st == self.prob.goal): 
            return start_node

        open_queue = [ start_node ]
        heapq.heapify(open_queue)
        
        closed_list = {}

        # print("open list: " + str(open_queue) + "\nclosed list: " + str(closed_list))

        while len(open_queue) != 0:
            n = heapq.heappop(open_queue)
            if n.st == self.prob.goal:
                return n
            child_nodes = self.expand(n)
            #print(child_nodes)
            for c in child_nodes:
                if c.st not in closed_list or c.cost < closed_list[c.st].cost:
                    closed_list[c.st] = c
                    c.manh=manhattan(c.st,self.prob.goal)
                    heapq.heappush(open_queue, c)
                    self.prob.maze.set(c.st[0],c.st[1],'0')
            print(self.prob.maze.maze)
            #print("open list: " + str(open_queue) + "\nclosed list: " + str(closed_list))
            #print('-------------------')
        return closed_list[self.prob.goal]
    

maze = Maze2D()
print(maze.is_start(6, 3))
print(maze.is_goal(12, 13))

prob3 = SimpleMaze((6, 3), (12, 13), maze)
solver2=Mazestar(prob3)
goalstar= solver2.search()
print(goalstar)

True
True
********************
*                  *
*     0            *
*    0S0           *
*     0            *
*                  *
*            *     *
*            *     *
*            *     *
*            *     *
*            *     *
*  ***********     *
*                  *
*           G      *
*                  *
*                  *
*                  *
*                  *
*                  *
********************

********************
*                  *
*     00           *
*    0S00          *
*     00           *
*                  *
*            *     *
*            *     *
*            *     *
*            *     *
*            *     *
*  ***********     *
*                  *
*           G      *
*                  *
*                  *
*                  *
*                  *
*                  *
********************

********************
*                  *
*     00           *
*    0S00          *
*    000           *
*     0            *
*            *     *
*

参考文献
- "Artificial Intelligence: A Modern Approach, 4th Global ed.", by Stuart Russell and Peter Norvig  
   http://aima.cs.berkeley.edu/index.html
- "ヒューリスティック探索入門", 陣内 佑  
   https://jinnaiyuu.github.io/pdf/textbook.pdf